In [4]:
import sqlalchemy as db
import pandas as pd
from sqlalchemy import ForeignKey 

In [5]:
engine = db.create_engine('sqlite:///base.sqlite') #Create test.sqlite automatically
connection = engine.connect()
metadata = db.MetaData()
emp = db.Table('Pays', metadata,
              db.Column('pays', db.String(255),primary_key=True),
              )
emp = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(255), nullable=False,primary_key=True),
              db.Column('pays', db.String(255), ForeignKey('Pays.pays')),
              db.Column('capacité', db.Integer()),
              db.Column('espace_enfant', db.Integer()),
              db.Column('service_rapide', db.Integer()),
              db.Column('parking', db.Integer, default=1),
              db.Column('accessibilite', db.Integer()),
              )
emp = db.Table('Employe', metadata,
              db.Column('id_employe',  db.Integer(),primary_key=True),
              db.Column('code_postal', db.String(200), ForeignKey('Restaurant.code_postal')),
            #   db.Column('id_superieur', db.Integer(), ForeignKey('Employe.id_employe')),
              db.Column('poste', db.String(200),nullable=False),
              db.Column('nom', db.String(200),nullable=False),
              db.Column('adresse', db.String(200), nullable=False),
              db.Column('note',db.Float())
              )
emp = db.Table('RIB', metadata,
              db.Column('id_employe',  db.Integer(),ForeignKey('Employe.id_employe'), nullable=False,primary_key=True),
              db.Column('iban', db.String(255),nullable=False),
              db.Column('bic', db.String(),nullable=False),
              db.Column('propriètaire', db.String(),nullable=False),
              db.Column('adresse', db.String(), nullable=False),
              )
emp = db.Table('Paie', metadata,
              db.Column('id_employe',  db.Integer(),ForeignKey('Employe.id_employe'), nullable=False,primary_key=True),
              db.Column('date', db.String(),nullable=False),
              db.Column('salaire_net', db.Float(),nullable=False),
              )
emp = db.Table('CarteMenu', metadata,
              db.Column('pays',  db.Integer(),ForeignKey('Pays.pays'), nullable=False,primary_key=True),
              db.Column('id_menu', db.Integer(),ForeignKey('Menu.id_menu'),primary_key=True,nullable=False),
              db.Column('salaire_net', db.Float(),nullable=False),
              )
emp = db.Table('Menu', metadata,
              db.Column('id_menu',  db.Integer(), nullable=False,primary_key=True),
              db.Column('boisson', db.String(),ForeignKey('Item.nom_item'),nullable=False),
              db.Column('plat', db.String(),ForeignKey('Item.nom_item'),nullable=False),
              db.Column('dessert', db.String(),ForeignKey('Item.nom_item'),nullable=False),
              db.Column('prix', db.Float(),nullable=False),
              )
emp = db.Table('Item', metadata,
              db.Column('nom_item',  db.String(), nullable=False,primary_key=True),
              db.Column('type', db.String(),nullable=False),
              db.Column('prix', db.Float(),nullable=False),
              )
emp = db.Table('CarteItem', metadata,
              db.Column('pays',  db.Integer(),ForeignKey('Pays.pays'), nullable=False,primary_key=True),
              db.Column('id_item', db.Integer(),ForeignKey('Item.nom_item'),primary_key=True,nullable=False),    
              )

emp = db.Table('PanierMenu', metadata,
              db.Column('id_bill',  db.Integer(),ForeignKey('Bill.id_bill'), nullable=False,primary_key=True),
              db.Column('id_menu', db.Integer(),ForeignKey('Menu.id_menu'),nullable=False),
              db.Column('quantité', db.Float(),nullable=False),
              )
emp = db.Table('PanierItem', metadata,
              db.Column('nom_item',  db.String(),ForeignKey('Item.nom_item'), nullable=False,primary_key=True),
              db.Column('id_bill', db.Integer(),ForeignKey('Bill.id_bill'),nullable=False),
              db.Column('quantité', db.Float(),nullable=False),
              )

emp = db.Table('Bill', metadata,
              db.Column('id_bill', db.Integer(),primary_key=True, nullable=False),
              db.Column('code_postal', db.String(255), ForeignKey('Restaurant.code_postal'),nullable=False),
              db.Column('id_vendeur', db.Integer(),ForeignKey('Employe.id_employe'),nullable=False),
              db.Column('born', db.Integer(),nullable=False),
              db.Column('moyen_paiment', db.String(),nullable=False),
              db.Column('prix_total', db.Float(),nullable=False),
)
emp = db.Table('Recette', metadata,
              db.Column('nom_item',  db.String(),ForeignKey('Item.nom_item'), nullable=False,primary_key=True),
              db.Column('nom_ingredient', db.String(),ForeignKey('Ingredient.nom_ingredient'),nullable=False,primary_key=True),
              db.Column('quantité', db.Integer(),nullable=False),
              )

emp = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(),nullable=False,primary_key=True),
              db.Column('cout', db.Float(),nullable=False),
)

emp = db.Table('Stock', metadata,
              db.Column('nom_ingredient', db.String(),ForeignKey('Ingredient.nom_ingredient'),nullable=False),
              db.Column('code_postal', db.String(),ForeignKey('Restaurant.code_postal'),nullable=False, primary_key=True),
              db.Column('quantité', db.Integer()),

)
metadata.create_all(engine) #Creates the table

In [ ]:
#Inserting record one by one
query = db.insert(emp).values(Id=1, name='naveen', salary=60000.00, active=True) 
ResultProxy = connection.execute(query)

/tmp/ipykernel_51561/293162416.py:3: SAWarning: Column 'Stock.code_postal' is marked as a member of the primary key for table 'Stock', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullable=True', and no explicit value is passed.  Primary key columns typically may not store NULL.
  ResultProxy = connection.execute(query)


CompileError: Unconsumed column names: name, active, salary, Id

In [ ]:
#Inserting many records at ones
query = db.insert(emp) 
values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
ResultProxy = connection.execute(query,values_list)


In [ ]:

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.shape